<a href="https://colab.research.google.com/github/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/01_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [249]:
import pandas as pd
import io

# Generar Archivos

In [250]:
import csv
def generar_csv(nombre_archivo, datos, separador=','):
    with open(nombre_archivo, 'w', newline='') as archivo:
        escritor = csv.writer(archivo, delimiter=separador)
        escritor.writerows(datos)


In [251]:
datos = [    ['id', 'nombre', 'edad'],
    [1, 'Ana', 25],
    [2, 'Carlos', 30],
    [3, 'Marta', 35]
]

generar_csv('archivo.csv', datos, separador=';')

Para inferir el tipo de separador de un archivo que Pandas va a leer, puedes utilizar la función `pd.read_csv()` con el parámetro `delimiter=None`. De esta manera, Pandas intentará inferir automáticamente el separador utilizado en el archivo CSV.

In [252]:
! head archivo.csv

id;nombre;edad
1;Ana;25
2;Carlos;30
3;Marta;35


# Delimiters

In [253]:
pd.read_csv('archivo.csv')

,id;nombre;edad
0,1;Ana;25
1,2;Carlos;30
2,3;Marta;35


In [254]:
pd.read_csv('archivo.csv', delimiter=';')

,id,nombre,edad
0,1,Ana,25
1,2,Carlos,30
2,3,Marta,35


In [255]:
def read_csv_with_delimiter(data_path:str, lines:int=5):
    # Solo usa las primeras 5 lineas para inferir el delimiter
    reader = pd.read_csv(data_path, sep = None, iterator = True, nrows=lines,
                         engine='python')
    delimiter = reader._engine.data.dialect.delimiter
    print(f'El delimiter encontrado fue \"{delimiter}\"')
    df = pd.read_csv(data_path, sep=delimiter)
    return df

En este código, se abre el archivo CSV con el modo de lectura 'r', y luego se utiliza la función `pd.read_csv()` con el parámetro `delimiter=None` para inferir automáticamente el separador utilizado en el archivo CSV. La propiedad dialect.delimiter del objeto retornado por la función `pd.read_csv(` se utiliza para obtener el separador inferido.

Luego, se reinicia el puntero de lectura del archivo CSV a la posición inicial utilizando el método `seek(0)`, y se utiliza nuevamente la función `pd.read_csv()` para leer los datos del archivo CSV utilizando el separador inferido como valor del parámetro sep.

In [256]:
read_csv_with_delimiter('archivo.csv',5)

El delimiter encontrado fue ";"


,id,nombre,edad
0,1,Ana,25
1,2,Carlos,30
2,3,Marta,35


# Datos de la nube

 utilizar la clase `io.StringIO` del módulo io de Python para crear un objeto de archivo en memoria. Esta clase proporciona una forma de crear un objeto de archivo similar a un archivo de entrada/salida que almacena datos en una cadena de texto en lugar de un archivo en disco.

In [257]:
# Generar datos de ejemplo
data = 'id,nombre,edad\n1,Ana,25\n2,Carlos,30\n3,Marta,35\n'
try:
    df = pd.read_csv(data)
except Exception as e:
    print(e)

[Errno 2] No such file or directory: 'id,nombre,edad\n1,Ana,25\n2,Carlos,30\n3,Marta,35\n'


In [258]:
# Leer archivo CSV simulado de S3 con Pandas
data_io = io.StringIO(data)
print(data_io)
df = pd.read_csv(data_io)
print(df)

   id  nombre  edad
0   1     Ana    25
1   2  Carlos    30
2   3   Marta    35


In [259]:
def read_csv_bytes_with_delimiter(data_stream, lines = 5):
    data_stream = io.StringIO(data_stream)
    reader = pd.read_csv(data_stream , sep = None, 
                         iterator = True, nrows=lines, engine='python')
    print(reader)
    delimiter = reader._engine.data.dialect.delimiter
    data_stream.seek(0) # Regresar al inicio del stream
    df = pd.read_csv(data_stream, sep=delimiter)
    return df

In [260]:
# Leer archivo CSV simulado de S3 con Pandas
read_csv_bytes_with_delimiter(data)

,id,nombre,edad
0,1,Ana,25
1,2,Carlos,30
2,3,Marta,35


In [261]:
data2 = 'i;d,nomb;re,ed;ad\n1,An;a,25\n2,Ca;rlos,30\n3,Ma;rta,35\n'
read_csv_bytes_with_delimiter(data2)

,i;d,nomb;re,ed;ad
0,1,An;a,25
1,2,Ca;rlos,30
2,3,Ma;rta,35


# Separadores dificiles


In [262]:
data3 = '''id;nombre,apellido;edad;correo
1;Juan,Pérez;35;jperez@gmail.com
2;María,García,Smith;42;mgarcia.smith@yahoo.com
3;Pedro,Álvarez;28;palvarez@hotmail.com'''

In [263]:
try:
    print(read_csv_bytes_with_delimiter(data3))
except Exception as e:
    print(e)

Error tokenizing data. C error: Expected 2 fields in line 3, saw 3



In [264]:
def read_csv_bytes_with_delimiter(data_stream, lines = 5, delimiter=None):
    data_stream = io.StringIO(data_stream)
    if not delimiter:
        reader = pd.read_csv(data_stream , sep = None, 
                            iterator = True, nrows=lines, engine='python')
        delimiter = reader._engine.data.dialect.delimiter
    print(delimiter)
    data_stream.seek(0) # Regresar al inicio del stream
    df = pd.read_csv(data_stream, sep=delimiter)
    return df

In [265]:
data2 = 'i;d,nomb;re,ed;ad\n1,An;a,25\n2,Ca;rlos,30\n3,Ma;rta,35\n'
read_csv_bytes_with_delimiter(data2)

,


,i;d,nomb;re,ed;ad
0,1,An;a,25
1,2,Ca;rlos,30
2,3,Ma;rta,35


In [266]:
read_csv_bytes_with_delimiter(data3,delimiter=';')

;


,id,"nombre,apellido",edad,correo
0,1,"Juan,Pérez",35,jperez@gmail.com
1,2,"María,García,Smith",42,mgarcia.smith@yahoo.com
2,3,"Pedro,Álvarez",28,palvarez@hotmail.com


# Errores en lineas

In [267]:
data4 = '''id;nombre,apellido;edad;correo
1;Juan,Pérez;35;jperez@gmail.com
2;María;García,Smith;42;mgarcia.smith@yahoo.com
3;Pedro,Álvarez;28;palvarez@hotmail.com'''


In [268]:
try:
    read_csv_bytes_with_delimiter(data4,delimiter=';')
except Exception as e:
    print(e)

;
Error tokenizing data. C error: Expected 4 fields in line 3, saw 5



In [269]:
def read_csv_bytes_with_delimiter_error(data_stream, lines = 5, delimiter=';'):
    data_stream = io.StringIO(data_stream)
    if not delimiter:
        reader = pd.read_csv(data_stream , sep = None, 
                            iterator = True, nrows=lines, engine='python', 
                             on_bad_lines='warn')
        delimiter = reader._engine.data.dialect.delimiter
    print(delimiter)
    data_stream.seek(0) # Regresar al inicio del stream
    df = pd.read_csv(data_stream, sep=delimiter, on_bad_lines='warn')
    return df

In [270]:
read_csv_bytes_with_delimiter_error(data4,delimiter=';')


;


b'Skipping line 3: expected 4 fields, saw 5\n'


,id,"nombre,apellido",edad,correo
0,1,"Juan,Pérez",35,jperez@gmail.com
1,3,"Pedro,Álvarez",28,palvarez@hotmail.com


# Encoding

In [271]:
with open("datos_enc.csv", "w", encoding="ISO-8859-1") as f:
    f.write("Universidad Autónoma de México,Universidad Nacional Autónoma de México\n")
    f.write("México,D.F.,Ciudad de México\n")

In [272]:
try:
    pd.read_csv("datos_enc.csv", delimiter="\n", header=None)
except Exception as e:
    print(e)

'utf-8' codec can't decode byte 0xf3 in position 15: invalid continuation byte


In [273]:
pd.read_csv("datos_enc.csv", encoding="ISO-8859-1", delimiter="\n", header=None)

,0
0,"Universidad Autónoma de México,Universidad Nac..."
1,"México,D.F.,Ciudad de México"


Aprovechemos para hacer mas flexible nuestras funciones

In [274]:
def read_csv_encoding(data_stream, lines = 5, delimiter=None, **kargs):
    if not delimiter:
        reader = pd.read_csv(data_stream , sep = None, 
                            iterator = True, nrows=lines, engine='python',**kargs)
        delimiter = reader._engine.data.dialect.delimiter
    print(delimiter)
    df = pd.read_csv(data_stream, sep=delimiter, **kargs)
    return df

In [275]:
read_csv_encoding("datos_enc.csv", delimiter="\n", encoding="ISO-8859-1")

,"Universidad Autónoma de México,Universidad Nacional Autónoma de México"
0,"México,D.F.,Ciudad de México"


In [276]:
read_csv_encoding("datos_enc.csv", delimiter="\n", encoding="ISO-8859-1", header=None)

,0
0,"Universidad Autónoma de México,Universidad Nac..."
1,"México,D.F.,Ciudad de México"


## Detectando encoding

Este código abre el archivo "datos_enc.csv" en modo binario, lee su contenido y utiliza la librería chardet para detectar la codificación. La función detect devuelve un diccionario que contiene información sobre la codificación detectada y su confianza. Finalmente, el código imprime la información obtenida.

In [277]:
import chardet

with open("datos_enc.csv", "rb") as f:
    result = chardet.detect(f.read())

encoding = result["encoding"]
confidence = result["confidence"]

print(f"La codificación detectada es {encoding} con una confianza del {confidence*100}%")


La codificación detectada es ISO-8859-1 con una confianza del 73.0%


In [278]:
def inferir_encoding(archivo:str):
    with open(archivo, "rb") as f:
        result = chardet.detect(f.read())

    encoding = result["encoding"]
    confidence = result["confidence"]

    print(f"La codificación detectada es {encoding} con una confianza del {confidence*100}%")
    return encoding, confidence

In [279]:
inferir_encoding("datos_enc.csv")

La codificación detectada es ISO-8859-1 con una confianza del 73.0%


('ISO-8859-1', 0.73)

## Detectando econding en StringIO

En este ejemplo, creamos un objeto io.StringIO a partir de una cadena codificada en "ISO-8859-1", pero lo decodificamos como "UTF-8" para simular una situación en la que la codificación es desconocida. Luego, obtenemos el contenido del objeto `io.StringIO` con el método `getvalue()`, lo codificamos como una cadena bytes utilizando "UTF-8" y utilizamos la librería chardet para detectar la codificación. Finalmente, imprimimos la codificación detectada.

In [280]:
# Creamos un objeto StringIO con un contenido en una codificación desconocida
contenido = "Hola, mundo! \n".encode("ISO-8859-1")
stringio = io.StringIO(contenido.decode("UTF-8"))

# Detectamos la codificación
contenido_str = stringio.getvalue()
resultado = chardet.detect(contenido_str.encode())

# Imprimimos la codificación detectada
print(f"La codificación detectada es {resultado['encoding']} con una confianza del {resultado['confidence']*100}%")


La codificación detectada es ascii con una confianza del 100.0%


# Regex (re) con pandas

# Generemos una lista aleatoria

**Tarea**: Investiga como funciona `pd.concat`. Revisa que tienes que hacer si quieres pegar `cols` o `rows`, y que pasa si los DataFrames tiene indices repetidos. 

Imagina que tienes que utilizar los emails de varias personas que trabajan en tu companya. Lamentablemente al no tener un sistema unificado, estan esparcidos por varios dominios. 

Sin embargo sabes que siempre son generados con `"xxxxxxx@dominio.yyy"`. Donde `x` puede ser numero, letra, punto o guion; mientras que `yyy` es un dominio valido como "com" o "es"  

Deberas desechar cualquier que no contenga el dominio, no trates de arreglar el email, pues no sabemos la razon del error. **Sin embargo, los emails que no tengan problemas en el dominio pueden ser rescatados**.  

Desecha si el dominio esta mal, arregla si el usuario antes del `@` esta bien.



In [289]:
import pandas as pd
import itertools
import random

# Generamos datos de prueba
nombres = ["Juan", "María", "Pedro", "Ana", "Luis", "Sara"]
apellidos = ["Pérez", "Gómez", "García", "Fernández", "Rodríguez", "González"]
dominios = ["gmail.com", "yahoo.com", "hotmail.com", "outlook.com", "yahoo.es"]
emails = []

# Generamos todas las combinaciones posibles entre nombres y apellidos
combinaciones = list(itertools.product(nombres, apellidos))
_df = []
random.seed(123543)
for i in range(100):
    # Seleccionamos una combinación de nombres y apellidos al azar
    nombre, apellido = random.choice(combinaciones)
    persona = f'{nombre} {apellido}'
    dominio = random.choice(dominios)
    email = f"{nombre.lower()}.{apellido.lower()}@{dominio}"
    if random.random() > .9:
        email = f"{nombre.lower()}{i}{apellido.lower()}@{persona}"
    elif random.random() < 0.3:
        email = f"{nombre.lower()}{apellido.lower()}@{persona}"
    _df.append(pd.DataFrame({"persona":[persona], "email": [email]}))

df = pd.concat(_df, ignore_index=True)
df.to_csv("clientes.csv", index=False)
df.head(20)

,persona,email
0,Ana Gómez,ana.gómez@yahoo.com
1,Luis Pérez,luis.pérez@gmail.com
2,Pedro García,pedrogarcía@Pedro García
3,Luis Gómez,luis.gómez@hotmail.com
4,Sara García,sara.garcía@outlook.com
5,María González,maría5gonzález@María González
6,María Fernández,maría6fernández@María Fernández
7,Juan García,juan.garcía@hotmail.com
8,Sara Gómez,sara8gómez@Sara Gómez
9,Sara Fernández,sara.fernández@gmail.com


Esta línea de código es una operación que se aplica sobre la columna `"email"` de un DataFrame llamado df, usando el método `apply()`. La operación se realiza en cada celda de la columna y utiliza una función `lambda`.

La función lambda toma cada valor x de la columna "email", lo convierte a una cadena de texto utilizando `str(x)`, y luego utiliza la función `re.findall()` para buscar todas las ocurrencias de la expresión regular regex_email en la cadena. La función `re.findall()` devuelve una lista con todas las ocurrencias encontradas en la cadena.

La expresión regular regex_email que se utiliza aquí es la que explicamos anteriormente y se utiliza para validar correos electrónicos.

Luego, se verifica si la lista de ocurrencias es mayor a cero, utilizando `len(re.findall(regex_email, str(x))) > 0`. Si esto es cierto, se toma el primer valor de la lista, utilizando `[0]`. Si no hay ocurrencias, se devuelve una cadena vacía, utilizando `else ""`.

En resumen, esta línea de código utiliza la expresión regular regex_email para validar los correos electrónicos en la columna "email" del DataFrame `df`. Si un correo electrónico es válido, se devuelve el correo electrónico como una cadena de texto. Si el correo electrónico no es válido, se devuelve una cadena vacía.

In [284]:
import re
# Leemos los datos
df = pd.read_csv("clientes.csv")

# Definimos la expresión regular para identificar direcciones de correo electrónico válidas
regex_email = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"

# Limpiamos la columna "email" utilizando la expresión regular
df["email"] = df["email"].apply(lambda x: re.findall(regex_email, str(x))[0] if len(re.findall(regex_email, str(x))) > 0 else "")

# Eliminamos los registros que no tienen una dirección de correo electrónico válida
df = df[df["email"] != ""]

# Guardamos los datos limpios
df
#df.to_csv("clientes_limpios.csv", index=False)

,persona,email
0,Ana Gómez,mez@yahoo.com
4,Sara García,a@outlook.com
6,Ana Gómez,mez@gmail.com
7,Luis Fernández,ndez@outlook.com
10,Pedro Fernández,ndez@yahoo.com
11,Ana Fernández,ndez@gmail.com
15,Pedro González,lez@gmail.com
17,Juan González,lez@hotmail.com
19,Sara Fernández,ndez@yahoo.es
20,Pedro García,a@hotmail.com


**Tarea**: Investiga como funciona `apply`. Revisa que tienes que hacer si quieres aplicarlo por columna o por row.

**Tarea**: Investiga como funciona `pd.concat`. Revisa que tienes que hacer si quieres pegar `cols` o `rows`, y que pasa si los DataFrames tiene indices repetidos. : Parece que estamos identificando los emails correctos, pero no los estamos guardando correctamente. Que esta pasando? Como lo puedes solucionar?